Idea: Test how to make calls including custom tooling
Initial Query: Send weather question to Claude with the tool definition

Tool Call: Claude recognizes this as a weather question and responds with a special "tool_use" message containing the location it extracted

Tool Execution: Execute actual weather function with this location and send results back to Claude

Response: Claude formats a natural language response using the weather data

In [ ]:
!pip install anthropic
!pip install python-dotenv

In [ ]:
import os
import anthropic
import json
#from dotenv import load_dotenv
from google.colab import userdata


In [ ]:
api_key = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(
    api_key=api_key
)


In [ ]:
# message = client.messages.create(
#     model="claude-3-7-sonnet-20250219",
#     max_tokens=1024,
#     messages=[
#         {"role": "user", "content": "Hello, Claude"}
#     ]
# )
# print(message)

In [ ]:
weather_tool = [
    {
        "name": "get_weather",
        "description": "Get the current weather",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and country, e.g. Paris, France"
                }
            },
            "required": ["location"]
        }
    }
]

In [ ]:
def is_european_location(location):
    european_countries = [
        "france", "germany", "uk", "spain", "italy", "netherlands",
        "belgium", "sweden", "norway", "finland", "denmark", "portugal",
        "ireland", "switzerland", "austria", "poland", "greece"
    ]

    location_lower = location.lower()

    for country in european_countries:
        if country in location_lower:
            return True

    european_cities = [
        "paris", "berlin", "london", "madrid", "rome", "amsterdam",
        "brussels", "stockholm", "oslo", "helsinki", "copenhagen",
        "lisbon", "dublin", "zurich", "vienna", "warsaw", "athens"
    ]

    for city in european_cities:
        if city in location_lower:
            return True

    return False


In [ ]:
def get_weather_data(location):
    # In a real implementation, we'd a weather API here

    is_europe = is_european_location(location)
    unit = "celsius" if is_europe else "fahrenheit"

    temp = 20 if unit == "celsius" else 68

    return {
        "location": location,
        "temperature": temp,
        "unit": unit,
        "condition": "Partly cloudy"
    }


In [ ]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    system="You are a helpful assistant. Use the weather tool when asked about weather.",
    messages=[
        {"role": "user", "content": "What's the weather like today in Dresden?"}
    ],
    tools=weather_tool
)

print(response)

Message(id='msg_012VwNSrnfhAgnU8vVbS3Ric', content=[TextBlock(citations=None, text='To get the current weather in Dresden, I can use the get_weather function. Let me fetch that information for you.', type='text'), ToolUseBlock(id='toolu_012phmYM5vmn34xL1CbSrcAn', input={'location': 'Dresden, Germany'}, name='get_weather', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=395, output_tokens=81))


In [ ]:
def get_weather_response(user_query):
    print("Step 1: Sending user query to Claude...")
    messages = [{"role": "user", "content": user_query}]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        system="You are a helpful assistant. Use the weather tool when asked about the weather.",
        messages=messages,
        tools=weather_tool
    )

    print("Response structure:", type(response))
    print("Response content length:", len(response.content))
    print("Content block types:", [block.type for block in response.content])

    # Check for tool_use blocks
    has_tool_call = False
    for content_block in response.content:
        if content_block.type == "tool_use":
            has_tool_call = True
            print("Step 2: Claude is calling the weather tool...")

            # Access the fields directly
            tool_name = content_block.name
            tool_input = content_block.input
            tool_id = content_block.id
            location = tool_input.get("location")

            print(f"Tool name: {tool_name}")
            print(f"Tool id: {tool_id}")
            print(f"Tool input: {tool_input}")

            # Add tool call to the conversation
            messages.append({
                "role": "assistant",
                "content": [content_block]
            })

            # Execute weather function
            print(f"Step 3: Getting weather data for {location}...")
            weather_data = get_weather_data(location)
            print("Weather data:", weather_data)

            # Here's the key fix: Format the result properly
            # Convert to a simple string representation
            result_string = json.dumps({
                "temperature": weather_data["temperature"],
                "unit": weather_data["unit"],
                "condition": weather_data["condition"]
            })

            print(f"Formatted result: {result_string}")

            # Add tool result to the conversation with the proper format
            messages.append({
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_id,
                        "result": result_string
                    }
                ]
            })

            print("Step 4: Getting Claude's final response...")
            final_response = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                max_tokens=1000,
                system="You are a helpful assistant. Use the weather tool when asked about the weather.",
                messages=messages,
                tools=weather_tool
            )

            return final_response.content[0].text

    # If no tool was used
    if not has_tool_call:
        print("Claude did not use the weather tool")
        return response.content[0].text

In [ ]:
get_weather_response("What's the weather like in Dresden today?")


Step 1: Sending user query to Claude...
Response structure: <class 'anthropic.types.message.Message'>
Response content length: 2
Content block types: ['text', 'tool_use']
Step 2: Claude is calling the weather tool...
Tool name: get_weather
Tool id: toolu_01L6GF6YAuRvnNRrJxax9UZs
Tool input: {'location': 'Dresden, Germany'}
Step 3: Getting weather data for Dresden, Germany...
Weather data: {'location': 'Dresden, Germany', 'temperature': 20, 'unit': 'celsius', 'condition': 'Partly cloudy'}
Formatted result: {"temperature": 20, "unit": "celsius", "condition": "Partly cloudy"}
Step 4: Getting Claude's final response...


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.2.content.0.tool_result.result: Extra inputs are not permitted'}}